In [3]:
from simtk import unit, openmm

# # Load test system
# from simtk.openmm import XmlSerializer
# with open('debug_repex/16_apo_0_hybrid_system.xml', 'r') as infile:
#     alchemical_system = XmlSerializer.deserialize(infile.read())
    
# Read initial coordinates
#from simtk.openmm.app import PDBFile
#pdbfile = PDBFile('initial.pdb')
#positions = pdbfile.positions

import pickle
# with open('debug_repex/16_apo_0_hybrid_positions.pickle', 'rb') as infile:
#     positions = pickle.load(infile)


import argparse
import os
# Read args
outdir = "/data/chodera/zhangi/perses_benchmark/repex/31/0/0/"
phase = 'apo'

In [6]:
# Read in htf
i = os.path.basename(os.path.dirname(outdir))
with open(os.path.join(outdir, f"{i}_{phase}.pickle"), 'rb') as f:
    htf = pickle.load(f)



In [8]:
alchemical_system = htf.hybrid_system
positions = htf.hybrid_positions


In [12]:
from perses.dispersed import feptasks

In [18]:
# Initialize compound thermodynamic states at different temperatures and alchemical states.
from openmmtools import alchemy, states
import numpy as np
n_replicas = 11
MAX_LAMBDA = 0.01
protocol = {
    'temperature': np.linspace(300, 300, n_replicas) * unit.kelvin,
    'lambda_torsions': np.linspace(0.0, MAX_LAMBDA, n_replicas),
    'lambda_angles': np.linspace(0.0, MAX_LAMBDA, n_replicas),
    'lambda_bonds': np.linspace(0.0, MAX_LAMBDA, n_replicas),
    'lambda_electrostatics_core': np.linspace(0.0, MAX_LAMBDA, n_replicas),
    'lambda_sterics_core': np.linspace(0.0, MAX_LAMBDA, n_replicas),
    'lambda_sterics_insert': np.linspace(0.0, MAX_LAMBDA, n_replicas),
    'lambda_sterics_delete': np.linspace(0.0, MAX_LAMBDA, n_replicas),
    'lambda_electrostatics_insert': np.linspace(0.0, MAX_LAMBDA, n_replicas),
    'lambda_electrostatics_delete': np.linspace(0.0, MAX_LAMBDA, n_replicas),
}
# Make sure AlchemicalState knows about some alchemical parameters
for name in ['lambda_sterics_core', 'lambda_electrostatics_core', 'lambda_sterics_delete', 'lambda_sterics_insert', 'lambda_electrostatics_delete', 'lambda_electrostatics_insert']:
    # Add the parameter
    setattr(alchemy.AlchemicalState, name, alchemy.AlchemicalState._LambdaParameter(name))
alchemical_state = alchemy.AlchemicalState.from_system(alchemical_system)
thermodynamic_states = states.create_thermodynamic_state_protocol(alchemical_system, protocol=protocol, composable_states=[alchemical_state])

# Initialize replica initial configurations.
sampler_states = [states.SamplerState(positions=positions, box_vectors=alchemical_system.getDefaultPeriodicBoxVectors()) for _ in thermodynamic_states]

for sampler_state, thermostate in zip(sampler_states, thermodynamic_states):
    feptasks.minimize(thermostate, sampler_state)

In [20]:
sampler_state.potential_energy

Quantity(value=-908721.4956418681, unit=kilojoule/mole)

In [21]:
# Propagate the replicas with Langevin dynamics.
from openmmtools import mcmc
n_steps = 250
langevin_move = mcmc.LangevinSplittingDynamicsMove(timestep=2.0*unit.femtosecond, n_steps=n_steps)
# langevin_move = mcmc.LangevinDynamicsMove(timestep=2.0*unit.femtosecond, n_steps=n_steps)

# Enable logging
import logging, sys
logging.basicConfig(level=logging.DEBUG, stream=sys.stdout)

# Run the parallel tempering simulation.
from openmmtools import multistate
n_iterations = 500
parallel_tempering = multistate.ReplicaExchangeSampler(mcmc_moves=langevin_move, number_of_iterations=n_iterations)
import tempfile
storage_path = f'{i}_{phase}.nc'
reporter = multistate.MultiStateReporter(storage_path, checkpoint_interval=n_iterations)
parallel_tempering.create(thermodynamic_states=thermodynamic_states, sampler_states=sampler_states, storage=reporter)
parallel_tempering.run(n_iterations=n_iterations)

Please cite the following:

        Friedrichs MS, Eastman P, Vaidyanathan V, Houston M, LeGrand S, Beberg AL, Ensign DL, Bruns CM, and Pande VS. Accelerating molecular dynamic simulations on graphics processing unit. J. Comput. Chem. 30:864, 2009. DOI: 10.1002/jcc.21209
        Eastman P and Pande VS. OpenMM: A hardware-independent framework for molecular simulations. Comput. Sci. Eng. 12:34, 2010. DOI: 10.1109/MCSE.2010.27
        Eastman P and Pande VS. Efficient nonbonded interactions for molecular dynamics on a graphics processing unit. J. Comput. Chem. 31:1268, 2010. DOI: 10.1002/jcc.21413
        Eastman P and Pande VS. Constant constraint matrix approximation: A robust, parallelizable constraint method for molecular simulations. J. Chem. Theor. Comput. 6:434, 2010. DOI: 10.1021/ct900463w
        Chodera JD and Shirts MR. Replica exchange and expanded ensemble simulations as Gibbs multistate: Simple improvements for enhanced mixing. J. Chem. Phys., 135:194110, 2011. DOI:10.1063/

ERROR:openmmtools.mcmc:Potential energy is NaN after 3 attempts of integration with move LangevinSplittingDynamicsMove Trying to reinitialize Context as a last-resort restart attempt...
ERROR:openmmtools.mcmc:Potential energy is NaN after 4 attempts of integration with move LangevinSplittingDynamicsMove
CRITICAL:openmmtools.multistate.multistatesampler:Propagating replica 0 at state 5 resulted in a NaN!
The state of the system and integrator before the error were saved in nan-error-logs


SimulationNaNError: Propagating replica 0 at state 5 resulted in a NaN!
The state of the system and integrator before the error were saved in nan-error-logs